# Lab 02: Text Processing

In this lab, you will pre-process the text data from your Program 1 assignment and practice computing at least two proximity measures using the sparse vectors you create from your data.

### Problem 1

Read in the data from train.dat and test.dat, split labels from text in the training data, and process the text in the training and test datasets into sparse matrices `train_data` and `test_data`. You may use and modify the code provided for you in the `Activity-data-3` example or use external libraries for this task. Note that both the training and test data must be in the same Euclidean space, i.e., each axis of the space should refer to the same token/word and there should be the same number of columns in the `train_data` and `test_data` matrices.

In [ ]:
python -m pip install nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

In [2]:
import string
import nltk
import re
import numpy as np
from scipy.sparse import csr_matrix
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter

def remove_punctuation(s):
    """
    Remove punctuation from a string
    """
    return s.translate(str.maketrans('','', string.punctuation))

def preprocess_line(line):
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    return [
                lemmatizer.lemmatize(stemmer.stem(w.lower())) 
                for w in remove_punctuation(line).split() 
                if len(w) > 0 and w.lower() not in stop_words
        ] 

def build_matrix(docs, idx, is_training):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    if len(idx) == 0:
        tid = 0
        nnz = 0
        for d in docs:
            nnz += len(set(d))
            for w in d:
                if w not in idx:
                    idx[w] = tid
                    tid += 1
    
    else:
        nnz = 0
        for d in docs:
            nnz += len([w for w in set(d) if w in idx])
    ncols = len(idx)
    # set up memory
    ind = np.zeros(nnz, dtype=int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d if is_training else [w for w in d if w in idx])
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

def process_data(fpath, idx=None, is_training=True):
    """
    This function processes the input data and returns the result.
    
    :param fpath: Input data to be processed
    :param idx: Optional dictionary of tokens
    :param is_training: Boolean flag indicating if the data is for training
    :return: csr_matrix containing the term-frequency values for the input data after processing
    """
     # Example processing: Read the data from fpath, split labels and features (if training data),
    # tokenize it, optionally filter tokens, apply stemming or lematization (but not both), 
    # and finally convert the data to a csr_matrix format by counting token frequencies in each doc.
    
    docs = []
    
    with open(fpath, 'r') as file:
        lines = file.readlines()
        for l in lines:
            if is_training:
                l = l[2:]
            processed = preprocess_line(l)
            docs.append(processed)

    data = build_matrix(docs, idx, is_training)
   
    return data
    
# Example usage
idx = {}
train_data = process_data('train.dat', idx=idx, is_training=True)
test_data = process_data('test.dat', idx=idx, is_training=False)
print(f"Training data number of features: {train_data.shape[1]}")
print(f"Test data number of features: {test_data.shape[1]}")
assert train_data.shape[1] == test_data.shape[1], "Train and test data do not have the same number of features"


Training data number of features: 74290
Test data number of features: 74290


### Problem 2

Now that you have the data, create efficient methods to compute at least one proximity function for samples in your data. In particular, to solve the k-NN problem you need to find the nearest neighbors within the training data (rows of `train_data`) for each of the test data points (rows of `test_data`). You may choose any proximity measures you would like. However, there are several requirements for this assignment:
1. You may not use any external libraries other than functions inherent in the `csr_matrix` data structure.
2. Your data matrix/vectors should not be changed from sparse to dense at any time during the computation.
3. You should return a list of pairs containing the rows in `train_data` that have non-zero proximity to the sample and their associated proximity values.

For best performance, computations should be vectorized whenever possible, i.e., you should compute vector-matrix or matrix-matrix operations rather than vector-vector operations.

In [5]:
def sparse_dot_product(row, x):
    x_i, x_v = x.indices, x.data
    r_i, r_v = row.indices, row.data

    dot = 0.0
    i = j = 0

    while i < len(x_i) and j < len(r_i):
        if x_i[i] == r_i[j]:
            dot += x_v[i] * r_v[j]
            i += 1
            j += 1
        elif x_i[i] < r_i[j]:
            i += 1
        else:
            j += 1

    return dot


def proximity(train_data, x):
    """
    This function computes the proximity of a given data point x to the training data.
    
    :param train_data: The training data in csr_matrix format
    :param x: The data point for which proximity is to be computed
    :return: list of pairs containing the rows in `train_data` that have non-zero proximity to the sample and their associated proximity values
    """
    proximities = []
    x_mag = sum(val * val for val in x.data) ** 0.5
    for i in range(train_data.shape[0]):
        row = train_data.getrow(i)
        dot_product = sparse_dot_product(row, x)
        row_mag = sum(val * val for val in row.data) ** 0.5

        if row_mag > 0 and x_mag > 0:
            sim = dot_product / (row_mag * x_mag)
            # each proximity is a pair, with the first value being the row number of the proximity and the second value the actual proximity
            proximities.append((i, float(sim)))
            
    proximities.sort(key=lambda pair: pair[1], reverse=True)
    return proximities

# Example usage 
x = test_data[0]
proximities = proximity(train_data, x)
print(f"Top 3 proximities out of {len(proximities)} using proximity function:", proximities[:3])

Top 3 proximities out of 102080 using proximity function: [(62678, 0.42874646285627205), (45692, 0.4117647058823529), (97924, 0.4115966043420212)]
